# Notebook Overview
First two sections of this Notebook are adapted from `Trajectron-plus-plus/experiments/nuScenes/evaluate.py` to run the Trajectron++ `eval_stg.predict()` function on the `../data/03_preprocessed/02_data.pkl` file produced by `01_Process-Data.ipynb`. Note that the Trajectron++ source code was modified to make `eval_stg.predict()` return additional information used for plotting. Additional information about these edits can be found in the `README.md`. Specifically, the following Trajectron++ files were modified:
* `Trajectron-plus-plus/trajectron/model/mgcvae.py`
* `Trajectron-plus-plus/trajectron/model/trajectron.py`

The latter portion of this Notebook is custom original plotting code to create and calculate useful plots/data for data analysis.

In [ ]:
import sys
import os
import dill
import json
import argparse
import torch
import numpy as np
import pandas as pd

sys.path.append("../../../trajectron")
#from tqdm import tqdm
from model.model_registrar import ModelRegistrar
from model.trajectron import Trajectron
import evaluation
import utils
from scipy.interpolate import RectBivariateSpline

from tqdm.notebook import tqdm

seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    
# python libraries
import numpy as np
import matplotlib.pyplot as plt

# increase default size matplotlib figures
from matplotlib import rcParams
rcParams['figure.figsize'] = (8, 5)

# Constants
Many of these constants replace values previously passed in as command line arguments.

In [ ]:
# Constants

# python evaluate.py
# --model models/int_ee
# --checkpoint=12
# --data ../processed/nuScenes_test_full.pkl
# --output_path results
# --output_tag int_ee
# --node_type VEHICLE
# --prediction_horizon 6

MODEL = '../../nuScenes/models/int_ee'
CHECKPOINT = 12
PREDICTION_HORIZON = 9
NODE_TYPE = 'VEHICLE'

INFILE = '../data/03_preprocessed/02_data.pkl'
BACKGROUND_IMG = '../data/misc/background.jpg'
OUTFILE_TEMPLATE = '../data/04_plots/%06d.jpg'

# Run Trajectron++ Model

In [ ]:
def load_model(model_dir, env, ts=100):
    model_registrar = ModelRegistrar(model_dir, 'cpu')
    model_registrar.load_models(ts)
    with open(os.path.join(model_dir, 'config.json'), 'r') as config_json:
        hyperparams = json.load(config_json)

    trajectron = Trajectron(model_registrar, hyperparams, None, 'cpu')

    trajectron.set_environment(env)
    trajectron.set_annealing_params()
    return trajectron, hyperparams

In [ ]:
with open(INFILE, 'rb') as f:
    env = dill.load(f, encoding='latin1')

eval_stg, hyperparams = load_model(MODEL, env, ts=CHECKPOINT)

if 'override_attention_radius' in hyperparams:
    for attention_radius_override in hyperparams['override_attention_radius']:
        node_type1, node_type2, attention_radius = attention_radius_override.split(' ')
        env.attention_radius[(node_type1, node_type2)] = float(attention_radius)

scenes = env.scenes

print("-- Preparing Node Graph")
for scene in tqdm(scenes):
    scene.calculate_scene_graph(env.attention_radius,
                                hyperparams['edge_addition_filter'],
                                hyperparams['edge_removal_filter'])

for ph in [PREDICTION_HORIZON]:
    print(f"Prediction Horizon: {ph}")
    max_hl = hyperparams['maximum_history_length']

    with torch.no_grad():
        ############### MOST LIKELY Z ###############
        eval_ade_batch_errors = np.array([])
        eval_fde_batch_errors = np.array([])

        print("-- Evaluating GMM Z Mode (Most Likely)")
        for scene in tqdm(scenes):
            timesteps = np.arange(scene.timesteps)

            # Trajectron++ Source Code was modified to change the behavior of
            # eval_stg.predict() to return additional data needed for plotting.
            # Specifically, the files
            # `Trajectron-plus-plus/trajectron/model/mgcvae.py` and
            # `Trajectron-plus-plus/trajectron/model/trajectron.py`
            # were modified to return multiple predictions and their corresponding
            # probabilities when run.
            
            # Default way of running Trajectron++ Model
            # (ignoring the additional return value since that was added)
            predictions,_ = eval_stg.predict(scene,
                                           timesteps,
                                           ph,
                                           num_samples=1,
                                           min_future_timesteps=ph,
                                           z_mode=True,
                                           gmm_mode=True,
                                           full_dist=False)  # This will trigger grid sampling
            
            # Modified version of running Trajectron++ Model so all
            # predictions and thier probabilities are returned.
            predictions_all,probs_all = eval_stg.predict(scene,    
                                      timesteps,    
                                      ph,    
                                      min_future_timesteps=ph,    
                                      z_mode=True,    
                                      gmm_mode=True,    
                                      all_z_sep=True,    
                                      full_dist=False)    

# General Definitions

In [ ]:
# Needed to directly index `predictions` dictionary (since vehicle objects are used as keys)
vehicles = dict(sorted([(int(node.id),node) for node in env.scenes[0].nodes],key=lambda x: x[0]))

In [ ]:
XI = 98.118385
XF = 169.524979
YI = 170.196945
YF = 226.819290
IMG = plt.imread(BACKGROUND_IMG)

# Plot config for most-likely plot
def mk_plot2D(fig,title=None,xlabel='X',ylabel='Y',lim=None):
    ax = fig.add_subplot(111)
    
    # set title if specified
    if title:
        ax.set_title(title)

    # tick params
    ax.tick_params(labelsize=14)
    
    # set labels
    ax.set_xlabel(xlabel, fontsize=16)
    ax.set_ylabel(ylabel, fontsize=16)

    # set axis limits if specified
    ax.set_xlim([XI,XF])
    ax.set_ylim([YF,YI])
    
    ax.imshow(IMG, extent=[XI,XF,YI,YF])
    
    x = []
    y = []
    ax.plot(x,y,'o',c='red',label='history',fillstyle='none',markersize=8,alpha=.5)
    ax.plot(x,y,'o',c='black',label='current',markersize=10)
    ax.plot(x,y,'o',c='green',label='true future',fillstyle='none',markersize=8,alpha=.5)
    ax.plot(x,y,'o',c='blue',label='prediction',fillstyle='none',markersize=8,alpha=.5)
    ax.legend(loc='lower right')

    return ax

# Plot config for distribution plot
def mk_plot2D_distribution(fig,title=None,xlabel='X',ylabel='Y',lim=None):
    ax = fig.add_subplot(111)
    
    # set title if specified
    if title:
        ax.set_title(title)

    # tick params
    ax.tick_params(labelsize=14)
    
    # set labels
    ax.set_xlabel(xlabel, fontsize=16)
    ax.set_ylabel(ylabel, fontsize=16)

    # set axis limits if specified
    ax.set_xlim([XI,XF])
    ax.set_ylim([YF,YI])
    
    ax.imshow(IMG, extent=[XI,XF,YI,YF])
    
    x = []
    y = []
    ax.plot(x,y,'o',c='red',label='history',fillstyle='none',markersize=8,alpha=.5)
    ax.plot(x,y,'o',c='black',label='current',markersize=10)
    #ax.plot(x,y,'o',c='green',label='true future',fillstyle='none',markersize=8,alpha=.5)
    ax.plot(x,y,'o',c='blue',label='prediction',markeredgecolor='none',markersize=8,alpha=.5)
    ax.legend(loc='lower right')

    return ax

# Plot Most-Likely Trajectory Prediction
Create plot frames showing the most-likely trajectory returned by Trajectron++. Outputs plot frames to `../data/04_plots` directory.

In [ ]:
# necessary to match Abbie/Shiloh's timestamps
# (they can't produced results until enough history has built up)
all_times = sorted(list(predictions.keys()))
times = all_times[7:] # start at timestep 8 to match other code

for t in tqdm(times):
    pred = predictions[t]
    fig = plt.figure(figsize=(8,6), dpi=80, facecolor='w', edgecolor='k')
    ax = mk_plot2D(fig)

    # bizarre timestamp label to match Abbie/Shiloh's timestamps
    ax.set_title(f't = {int(np.floor((t-1)/3))}', fontsize=16)
    
    for veh in pred:
        t_to_idx = t - veh.first_timestep
        length = len(veh.data.data)
        
        start = 0 if t_to_idx < PREDICTION_HORIZON else t_to_idx - PREDICTION_HORIZON
        stop = length if t_to_idx + PREDICTION_HORIZON + 1 > length else t_to_idx + PREDICTION_HORIZON + 1
        
        # History
        x,y = veh.data.data[start:t_to_idx,0:2].T
        ax.plot(x,y,'o',c='red',fillstyle='none',markersize=8,alpha=.5)

        # Future
        x,y = veh.data.data[t_to_idx+1:stop,0:2].T
        ax.plot(x,y,'o',c='green',fillstyle='none',markersize=8,alpha=.5)

        # Prediction
        x,y = pred[veh][0][0].T
        ax.plot(x,y,'o',c='blue',fillstyle='none',markersize=8,alpha=.5)
        
        # Current
        x,y = veh.data.data[t_to_idx,0:2].T
        ax.plot(x,y,'o',c='black',markersize=10)

    plt.savefig(OUTFILE_TEMPLATE % (t-times[0]))
    plt.close()
    
    # Comment out above two lines and uncomment below two lines to get the first 20
    # graphs in the Notebook. Useful for sampling graph edits before writing a whole
    # folder of files.
    
    # if t > 20:
    #     break

plt.close(fig='all')

# Plot Trajectory Prediction Distribution
Create plot frames showing the distribution of trajectories returned by Trajectron++. Outputs plot frames to `../data/04_plots` directory.

**Warning:** If run immediately after previous cell, it will overwrite the results from that cell.

In [ ]:
# necessary to match Abbie/Shiloh's timestamps
# (they can't produced results until enough history has built up)
all_times = sorted(list(predictions.keys()))
times = all_times[7:] # start at timestep 8 to match other code

for t in tqdm(times):
    pred = predictions_all[t]
    prob = probs_all[t]
    fig = plt.figure(figsize=(8,6), dpi=80, facecolor='w', edgecolor='k')
    ax = mk_plot2D_distribution(fig)
    
    # bizarre timestamp label to match Abbie/Shiloh's timestamps
    ax.set_title(f't = {int(np.floor((t-1)/3))}', fontsize=16) 
    
    for veh in pred:
        t_to_idx = t - veh.first_timestep
        length = len(veh.data.data)
        
        start = 0 if t_to_idx < PREDICTION_HORIZON else t_to_idx - PREDICTION_HORIZON
        stop = length if t_to_idx + PREDICTION_HORIZON + 1 > length else t_to_idx + PREDICTION_HORIZON + 1
        
        # History
        x,y = veh.data.data[start:t_to_idx,0:2].T
        ax.plot(x,y,'o',c='red',fillstyle='none',markersize=8,alpha=.5)

        # Prediction Distribution
        pred_veh = pred[veh][0]
        prob_veh = prob[veh][0]
        
        num_sig = np.count_nonzero(prob_veh >= .005)
        arg_order = np.argsort(prob_veh)[::-1]
        selected = arg_order[0:num_sig]
        
        plot_traj = pred_veh[selected]
        
        for curr in plot_traj:
            x,y = curr.T
            ax.plot(x,y,'o',c='blue',markeredgecolor='none',markersize=8,alpha=.075)
        
        # Disabled plotting Future and Prediction (Most) to unclutter view
        
        # Future
        x,y = veh.data.data[t_to_idx+1:stop,0:2].T
        #ax.plot(x,y,'o',c='green',fillstyle='none',markersize=8,alpha=1)
        
        # Prediction (Most)
        x,y = pred[veh][0][0].T
        #ax.plot(x,y,'o',c='cyan',fillstyle='none',markersize=8,alpha=1)
        
        # Current
        x,y = veh.data.data[t_to_idx,0:2].T
        ax.plot(x,y,'o',c='black',markersize=10)

    plt.savefig(OUTFILE_TEMPLATE % (t-times[0]))
    plt.close()
    
    # Comment out above two lines and uncomment below two lines to get the first 20
    # graphs in the Notebook. Useful for sampling graph edits before writing a whole
    # folder of files.
    
    # if t > 20:
    #     break

plt.close(fig='all')

# Calculate ADE/FDE
Calculate average displacement error (ADE) and final displacement error (FDE) values for presentation.

In [ ]:
# average displacement error (ADE)
# final displacement error (FDE)

for t in tqdm(sorted(list(predictions.keys()))):
    pred = predictions[t]

    ADE = []
    FDE = []
    
    for veh in pred:
        t_to_idx = t - veh.first_timestep
        length = len(veh.data.data)
        
        start = 0 if t_to_idx < PREDICTION_HORIZON else t_to_idx - PREDICTION_HORIZON
        stop = length if t_to_idx + PREDICTION_HORIZON + 1 > length else t_to_idx + PREDICTION_HORIZON + 1
        
        # Future
        pos_f = veh.data.data[t_to_idx+1:stop,0:2].T

        # Prediction
        pos_p = pred[veh][0][0].T
        
        assert pos_f.shape == pos_p.shape
        
        dist = np.linalg.norm(pos_f - pos_p, axis=0)
        
        ADE.append(np.average(dist))
        FDE.append(dist[-1])
    
print('ADE:',np.average(ADE))
print('FDE:',np.average(FDE))

# FFMPEG
These commands run `ffmpeg` in the `../data/04_plots` directory to create video files from all the individual plots. Google Slides cannot play videos at 8fps, so the second command converts the video to a more standard 30fps.

In [ ]:
!ffmpeg -y -framerate 8 -i ../data/04_plots/%06d.jpg ../data/04_plots/video_8fps.mp4

In [ ]:
!ffmpeg -y -i ../data/04_plots/video_8fps.mp4 -r 30 ../data/04_plots/video.mp4